In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from optimizer_supervised import Optimizer
from Ising import Ising
from model import TransformerModel

In [3]:
def gpu_setup():
    # Setup for PyTorch:
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        print("PyTorch is using GPU {}".format(torch.cuda.current_device()))
    else:
        torch_device = torch.device("cpu")
        print("GPU unavailable; using CPU")

In [4]:
gpu_setup()
torch.set_default_device("cuda")

PyTorch is using GPU 0


In [5]:
system_sizes = torch.arange(2, 21, 2, device="cpu").reshape(-1, 1)
system_sizes

tensor([[ 2],
        [ 4],
        [ 6],
        [ 8],
        [10],
        [12],
        [14],
        [16],
        [18],
        [20]])

In [6]:
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1718580740865/work/aten/src/ATen/native/TensorShape.cpp:3675.)
  return func(*args, **kwargs)
/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


In [7]:
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 1000
param_range = None
point_of_interest = None
use_SR = False

In [8]:
# Small allocation for model parameters, layers, etc.
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
testmodel.cuda()

TransformerModel(
  (pos_encoder): TQSPositionalEncoding1D(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=32, out_features=32, bias=True)
          (linear_Q): Linear(in_features=32, out_features=32, bias=True)
          (linear_K): Linear(in_features=32, out_features=32, bias=True)
          (linear_V): Linear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=32, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=32, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
    )


In [10]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

## Following .forward

In [11]:
H = Hamiltonians[7]
print(H.system_size)

tensor([16])


In [12]:
spins = H.basis

In [13]:
testmodel.set_param(system_size=H.system_size, param=torch.tensor([1.0], device="cuda"))

In [14]:
res = testmodel.forward(spins)
res

[tensor([[[-0.5700, -0.8337],
          [-0.5700, -0.8337],
          [-0.5700, -0.8337],
          ...,
          [-0.5700, -0.8337],
          [-0.5700, -0.8337],
          [-0.5700, -0.8337]],
 
         [[-0.5593, -0.8477],
          [-0.5593, -0.8477],
          [-0.5593, -0.8477],
          ...,
          [-0.4986, -0.9349],
          [-0.4986, -0.9349],
          [-0.4986, -0.9349]],
 
         [[-0.5328, -0.8842],
          [-0.5328, -0.8842],
          [-0.5328, -0.8842],
          ...,
          [-0.4639, -0.9911],
          [-0.4639, -0.9911],
          [-0.4639, -0.9911]],
 
         ...,
 
         [[-0.5647, -0.8406],
          [-0.5647, -0.8406],
          [-0.5647, -0.8406],
          ...,
          [-0.5110, -0.9161],
          [-0.5110, -0.9161],
          [-0.5110, -0.9161]],
 
         [[-0.5470, -0.8644],
          [-0.5470, -0.8644],
          [-0.5173, -0.9067],
          ...,
          [-0.5224, -0.8992],
          [-0.4894, -0.9493],
          [-0.4894, -0.9493

In [15]:
del res

In [16]:
del testmodel

In [17]:
import gc

gc.collect()

565

In [18]:
torch.cuda.empty_cache()

In [14]:
src = testmodel.wrap_spins(spins)

In [15]:
testmodel.src_mask = testmodel._generate_square_subsequent_mask(src.size(0)).to(
    src.device
)
testmodel.src_mask

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0.,

In [16]:
system_size = src[
    : testmodel.n_dim, 0, testmodel.phys_dim : testmodel.phys_dim + testmodel.n_dim
].diag()
system_size

tensor([2.7726], device='cuda:0')

In [17]:
system_size = system_size.exp().round().to(torch.int64)
system_size

tensor([16], device='cuda:0')

In [18]:
import math

In [19]:
src = testmodel.encoder(src) * math.sqrt(testmodel.embedding_size)
src

tensor([[[-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698],
         [-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698],
         [-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698],
         ...,
         [-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698],
         [-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698],
         [-1.4493,  1.2511,  0.7615,  ..., -0.6516, -1.4590, -0.8698]],

        [[-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424],
         [-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424],
         [-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424],
         ...,
         [-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424],
         [-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424],
         [-0.0892, -0.2254,  0.2675,  ..., -0.2129,  0.3032, -0.1424]],

        [[ 0.1982, -0.0404, -0.3834,  ..., -0.1147, -0.5085,  0.5536],
         [ 0.1982, -0.0404, -0.3834,  ..., -0

There's a problem if GPU memory usage increases at this point; this operation should be in-place.


In [20]:
src = testmodel.pos_encoder(src)
src

tensor([[[-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660],
         [-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660],
         [-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660],
         ...,
         [-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660],
         [-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660],
         [-1.4560,  1.2795,  0.7603,  ..., -0.6402, -1.4689, -0.8660]],

        [[-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544],
         [-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544],
         [-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544],
         ...,
         [-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544],
         [-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544],
         [-0.1021, -0.2299,  0.2635,  ..., -0.2287,  0.3352, -0.1544]],

        [[ 0.1982,  0.9596, -0.3834,  ...,  0.8853, -0.5085,  1.5536],
         [ 0.1982,  0.9596, -0.3834,  ...,  0

In [21]:
output = testmodel.transformer_encoder(src, testmodel.src_mask)
output

tensor([[[ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149],
         [ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149],
         [ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149],
         ...,
         [ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149],
         [ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149],
         [ 1.3218, -0.5965, -1.0745,  ...,  0.3264, -0.4246, -1.3149]],

        [[ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868],
         [ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868],
         [ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868],
         ...,
         [ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868],
         [ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868],
         [ 0.9145, -0.9515, -0.9850,  ..., -0.0988, -0.5072, -0.3868]],

        [[ 0.3780, -1.1240, -1.0349,  ..., -0.2109, -0.7638,  1.0329],
         [ 0.3780, -1.1240, -1.0349,  ..., -0